<a href="https://colab.research.google.com/github/Enkrumah14/mannyNkrumahGenAi/blob/main/HW4/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#author : Manny Nkrumah
#file : Problem1.ipynb
#assignment # : 4
#date : 11/04/24
#description : GAN architecture bedroom model

!pip install patool
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import os
import glob
from PIL import Image
import time
from tqdm import tqdm
import requests
import patoolib

# Hyperparameters
EPOCHS = 10
BATCH_SIZE = 256
noise_dim = 100
num_examples_to_generate = 16
generator_lr = 1e-4
discriminator_lr = 1e-4

# Links
archive_path = "archive.zip"
output_dir = "dataset"
url = "https://www.kaggle.com/api/v1/datasets/download/jhoward/lsun_bedroom"  # Dataset URL

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Download dataset archive in chunks
with requests.get(url, stream=True) as r:
    r.raise_for_status()  # Check for download errors
    with open(archive_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print(f"Dataset downloaded to: {archive_path}")

# Extract archive using patoolib
try:
    patoolib.extract_archive(archive_path, outdir=output_dir, program="unzip")
except patoolib.util.PatoolError:
    try:
        patoolib.extract_archive(archive_path, outdir=output_dir, program="7z")
    except patoolib.util.PatoolError:
        print("Extraction failed: check if the file is corrupted or in a supported format.")

# Load image file paths
image_files = glob.glob(f"{output_dir}/**/*.jpg", recursive=True)
if not image_files:
    raise ValueError("No image files found. Please check the output_dir path.")
print(f"Found {len(image_files)} image files.")

# Define a function to load and preprocess images
def load_and_preprocess_image(path):
    """Loads an image from the given path and preprocesses it."""
    image = tf.io.read_file(path)  # Read the image file
    image = tf.image.decode_jpeg(image, channels=3)  # Decode JPEG image
    image = tf.image.resize(image, [64, 64])  # Resize to 64x64
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

# Create a TensorFlow dataset and preprocess images
dataset = tf.data.Dataset.from_tensor_slices(image_files)
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)  # Apply the preprocessing function
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)

# Define generator model
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Define discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Loss functions for GAN training
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(generator_lr)
discriminator_optimizer = tf.keras.optimizers.Adam(discriminator_lr)

# Define training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Define training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for image_batch in tqdm(dataset, desc=f"Epoch {epoch+1}/{epochs}"):
            train_step(image_batch)
        generate_and_save_images(generator, epoch + 1, seed)
        print(f'Time for epoch {epoch + 1} is {time.time()-start:.2f} sec')

# Generate and save images during training
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :] * 0.5 + 0.5)
        plt.axis('off')
    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

# Main execution
generator = make_generator_model()
discriminator = make_discriminator_model()
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

train(dataset, EPOCHS)  # Start training


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 2.3 MB/s eta 0:00:00


KeyboardInterrupt: 